Integrate the cartographic projections and coordinate transformations library

In [ ]:
!pip install pyproj pandas

In [ ]:
import pandas as pd
from pyproj import Proj, transform

file_path = 'calles_SER_2024.xlsx'
data = pd.read_excel(file_path)

utm_proj = Proj(proj='utm', zone=30, datum="WGS84")
latlon_proj = Proj(proj="latlong", datum="WGS84")

def convert_utm_to_latlon(x,y):
  lon, lat = transform(utm_proj, latlon_proj, x, y)
  return lat, lon

data[['latitude', 'longitude']] = data.apply(
    lambda row: pd.Series(convert_utm_to_latlon(row['gis_x'], row['gis_y'])), axis=1
)

output_file = "coordinates_calles_SER_2024.xlsx"
data = pd.read_excel(output_file)

<ipython-input-2-8c9c29112c29>:11: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  lon, lat = transform(utm_proj, latlon_proj, x, y)


In [ ]:
data.to_excel(output_file, index=False)
print(f"Converted coordinates saved to {output_file}")

Converted coordinates saved to coordinates_calles_SER_2024.xlsx


### Convert into JSON data

In [7]:
import pandas as pd
file_path = 'coordinates_calles_SER_2024.xlsx'

data = pd.read_excel(file_path)

json_file = 'output.json'
data.to_json(json_file, orient='records', lines=False)
print(f"Json file created: {json_file}")

Json file created: output.json


In [ ]:
import json
import pandas as pd

with open('output.json', 'r') as f:
    data = json.load(f)

geojson = {
    "type": "FeatureCollection",
    "features": []
}

for row in data:
    feature = {
        "type": "Feature",
        "geometry": {
            "type": "Point",
            "coordinates": [row['longitude'], row['latitude']]
        },
        "properties": {
            "district": row.get('distrito', ''),
            "neighborhood": row.get('barrio', ''),
            "street": row.get('calle', ''),
            "property_number": row.get('numero_finca', ''),
            "color": row.get('color', ''),
            "available spots": row.get('numero_plazas', ''),
        }
    }
    geojson["features"].append(feature)

with open('output.geojson', 'w') as f:
    json.dump(geojson, f, indent=2)

print("GeoJSON file created: output.geojson")


GeoJSON file created: output.geojson
